In [1]:
#07-09-2022 
#06-09-2022 parcing
#05-09-2022 dct -> dict, optim
#04-09-2022 
#02-09-2022 разработка классов
#01-09-2022 PHC 8000 import importlib 
#25-08-2022 Создна копия для SUNLIGHT
#25-08-2022 Для цек БК и ПДК
#07-07-2022 0 обработка нефти 
#06-07-2022 серебро https://miuz.ru/catalog/filter/offers_metall-is-serebro/page=2/
#16-06-2022 Для Алексея Руб-Из-Сап
#24-05-2022 Запуск парсинга result is 'mius240522_0-4154.xlsx'

## Код для перезапуска модулей и классов

In [21]:
import importlib 

In [34]:
import classes_parsing, classes_site #https://stackoverflow.com/questions/6946376/how-to-reload-a-class-in-python-shell

In [35]:
importlib.reload(classes_parsing)

<module 'classes_parsing' from 'C:\\Users\\специалист ао\\mypandas\\work\\CDO2030\\Parsing_NKO\\classes_parsing.py'>

In [36]:
importlib.reload(classes_site)

<module 'classes_site' from 'C:\\Users\\специалист ао\\mypandas\\work\\CDO2030\\Parsing_NKO\\classes_site.py'>

In [37]:
from classes_parsing import Dtask,Dicts

In [38]:
from classes_site import Site

## Модули

In [2]:
import datetime
import json
import os
import re
import requests
from requests import Request, Session
from bs4 import BeautifulSoup
import pandas as pd

from classes_parsing import Dtask,Dicts
from classes_site import Site

# Разработка класса Site

In [45]:
class Site():
    '''v0003'''
    folderres = 'results/'
    
    def __init__(self):
        '''parametr: '''
        #self.dict = dct
        print ('create')#self.dict['domain'])
     
    def mysoup(self,urls):
        response = requests.get(urls)
        return BeautifulSoup(response.text, 'lxml')  
  
    def getlinks_sunl(self,urls):
        quotes = self.mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")
        res = [q.get("href") for q in quotes]
        print(len(res), 'links is ready ' , res[0], '...', res[-1])
        return res
        
        
    def genlinks_sunl(self, task):
        '''task: classes_parsing.Dtask'''
        yield task.dict['urls0']
       # 'urlslast':'https://sunlight.net/catalog/diamonds-all/page-104/'
        text = task.dict['urlslast'].strip().split('/')[-2]
        n = int(re.search(r'\d+',text)[0])
        ch = text[re.search(r'\d+',text).span()[0]-1]
        urls = '/'.join(task.dict['urlslast'].strip().split('/')[0:-2])
        for i in range(2,n):
            pages = f'/page{ch}{i}/'
            #pages = f'&page={i}'#без вставки
            yield urls+pages
                  
        
    def getlinks(self, task)->None:#
        '''create list links and modify object task'''
        genlink = self.genlinks_sunl(task)
        links = []
        i = 0
        for l in genlink:
            print(i, l)
            links += self.getlinks_sunl(l)
            i+=1
        task.dict['links'] = links
        
        
        
    def parcing(self, task):
        def savereserve_csv(data):
            datestr = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
            filename = self.folderres + '/_reserve_' + task.dict['name'] + datestr + '.csv'
            data.to_csv(filename)
            return filename
        
            
        data = pd.DataFrame(columns=['h1','art','price','price2','gold','gold2','weight','gems','gems2','url'])
        domain = task.dict['domain']
        links = task.dict['links']               
        print('парсинг  ', len(links))
        for i in range(0,len(links)):
            try:
                row = i
                urlcard = domain + links[i]
                response = requests.get(urlcard)
                card = BeautifulSoup(response.text, 'lxml')
                #card = mysoup(urlcard)
                data.loc[row, 'url'] = urlcard
                data.loc[row, 'h1'] = card.find("h1").text.strip()
                data.loc[row, 'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
                data.loc[row, 'price'] = card.find('div',
                                                   class_="supreme-product-card__price-discount-price").text.strip().replace(
                    '\u202f', '').replace('\xa0', '')
                data.loc[row, 'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace(
                    '\u202f', '').replace('\xa0', '')
                a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
                data.loc[row, 'gold'] = ';'.join([x.text for x in a])
                data.loc[row, 'd1'] = "&".join(
                    [t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
                data.loc[row, 'd0'] = "&".join(
                    [t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
                # print(i, urlcard)
            except (AttributeError, TypeError) as e:
                print('error', e)
            except IndexError:
                print('IndexError')
            except Exception as e:
                print('ошибка:', e)
                input('жду')
            print(i, urlcard)
            if i%100 == 0:
                print('saved', i, savereserve_csv(data))
            
        lastfile = savereserve_csv(data)
        print('saved last', i, lastfile)
        task.dict['datestr'] = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        task.dict['parcing'] = lastfile
        return data



      

## Тесты модуля разработки

In [39]:
dtask = {'name':'SL_diamonds',
         'domain': 'https://sunlight.net/',
         #'links':[],
         'urls0':'https://sunlight.net/catalog/diamonds-all.html',
         'urlslast':'https://sunlight.net/catalog/diamonds-all/page-4/',
         'descr':''}
        

In [40]:
task = Dtask(dtask)

установка : SL_diamonds2022-09-07-17-29


In [41]:
#task = Dtask({'name':'SL_diamonds2'})

In [42]:
#task.load()

In [48]:
task.print()

task name: SL_diamonds2022-09-07-17-29
links len =  180 

{'name': 'SL_diamonds2022-09-07-17-29',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-3/',
'descr': '',
'datestr0': '2022-09-07-17-29',
'datestr': None,
}


In [47]:
#task.dict['urlslast']= 'https://sunlight.net/catalog/diamonds-all/page-3/'

In [288]:
#task.save()

не правильный словарь
dict записан:  d_SL_diamonds2022-09-07-08-24_2022-09-07-08-25.json


In [8]:
#task.load()

In [44]:
site = Site()

create


In [49]:
site.getlinks(task)

0 https://sunlight.net/catalog/diamonds-all.html
60 links is ready  /catalog/ring_165495.html ... /catalog/earring_146527.html
1 https://sunlight.net/catalog/diamonds-all/page-2/
60 links is ready  /catalog/pendants_58553.html ... /catalog/earring_313143.html


In [50]:
task.print()

task name: SL_diamonds2022-09-07-17-29
links len =  120 

{'name': 'SL_diamonds2022-09-07-17-29',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-3/',
'descr': '',
'datestr0': '2022-09-07-17-29',
'datestr': None,
}


In [51]:
site.parcing(task)

парсинг   120
0 https://sunlight.net//catalog/ring_165495.html
saved 0 results//_reserve_SL_diamonds2022-09-07-17-292022-09-07-17-31.csv
1 https://sunlight.net//catalog/ring_146537.html
2 https://sunlight.net//catalog/ring_57830.html
3 https://sunlight.net//catalog/ring_47720.html
4 https://sunlight.net//catalog/ring_108790.html
5 https://sunlight.net//catalog/ring_109474.html
6 https://sunlight.net//catalog/ring_108796.html
7 https://sunlight.net//catalog/ring_152015.html
8 https://sunlight.net//catalog/ring_56703.html
9 https://sunlight.net//catalog/ring_48271.html
10 https://sunlight.net//catalog/earring_108798.html
11 https://sunlight.net//catalog/ring_238293.html
12 https://sunlight.net//catalog/ring_283369.html
13 https://sunlight.net//catalog/ring_69240.html
14 https://sunlight.net//catalog/ring_56676.html
15 https://sunlight.net//catalog/earring_48270.html
16 https://sunlight.net//catalog/earring_47719.html
17 https://sunlight.net//catalog/ring_100269.html
18 https://sunlight.n

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url,d1,d0
0,Золотое кольцо «Бриллианты Якутии»,Артикул: 165495,4990₽,24950₽,Артикул: 165495;Кольцо «Бриллианты Якутии» с 2...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_165495.html,3101-K5R-04&РОССИЯ&Розовое золото&Бриллиант&0....,Бриллианты Якутии&Бриллианты Якутии
1,Золотое кольцо «Бриллианты Якутии»,Артикул: 146537,11990₽,39967₽,Артикул: 146537;Кольцо «Бриллианты Якутии» с 1...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_146537.html,70669-K5R-01&РОССИЯ&Розовое золото&Бриллиант&1...,Бриллианты Якутии&Бриллианты Якутии
2,Золотое кольцо «Бриллианты Якутии»,Артикул: 57830,19990₽,66633₽,Артикул: 57830;Кольцо «Бриллианты Якутии» с 1 ...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_57830.html,0490-K5R-02&РОССИЯ&Розовое золото&Бриллиант&1....,Бриллианты Якутии&Бриллианты Якутии
3,Золотое кольцо «Бриллианты Якутии»,Артикул: 47720,10990₽,36633₽,Артикул: 47720;Кольцо «Бриллианты Якутии» с 1 ...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_47720.html,"0496-K5R-01&РОССИЯ&Розовое золото&Бриллиант&1,...",Бриллианты Якутии&Бриллианты Якутии
4,Золотое кольцо с бриллиантами,Артикул: 108790,9990₽,33300₽,"Артикул: 108790;Кольцо с 55 бриллиантами, огра...",NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_108790.html,70521-K5R-01&РОССИЯ&Розовое золото&Бриллиант&1...,Бриллианты Якутии&Бриллианты. Классика
...,...,...,...,...,...,...,...,...,...,...,...,...
115,Золотые серьги с топазами и бриллиантами,Артикул: 54694,24990₽,49980₽,"Артикул: 54694;Серьги с 20 бриллиантами, огран...",NaN,NaN,NaN,NaN,https://sunlight.net//catalog/earring_54694.html,9347-C5R-08&РОССИЯ&Розовое золото&Бриллиант; Т...,SERGEY GRIBNYAKOV&ЗОЛОТО. ТОПАЗЫ
116,Золотые серьги с изумрудами и бриллиантами,Артикул: 69297,33990₽,67980₽,"Артикул: 69297;Серьги с 26 бриллиантами, огран...",NaN,NaN,NaN,NaN,https://sunlight.net//catalog/earring_69297.html,9945-C5R-05&РОССИЯ&Розовое золото&Бриллиант; И...,SERGEY GRIBNYAKOV&Золото. Изумруды
117,Золотая подвеска «Бриллианты Якутии»,Артикул: 108794,3990₽,39900₽,Артикул: 108794;Подвеска «Бриллианты Якутии» с...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/pendants_108794....,70530-P5R-01&РОССИЯ&Розовое золото&Бриллиант&0...,Бриллианты Якутии&Бриллианты Якутии
118,Золотые серьги «Бриллианты Якутии»,Артикул: 71405,71990₽,143980₽,Артикул: 71405;Серьги «Бриллианты Якутии» с 2 ...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/earring_71405.html,3598-C5R-01&РОССИЯ&Розовое золото&Бриллиант&2....,Бриллианты Якутии&Бриллианты Якутии


In [52]:
task.print()

task name: SL_diamonds2022-09-07-17-29
links len =  120 

{'name': 'SL_diamonds2022-09-07-17-29',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-3/',
'descr': '',
'datestr0': '2022-09-07-17-29',
'datestr': '2022-09-07-17-34',
'parcing': 'results//_reserve_SL_diamonds2022-09-07-17-292022-09-07-17-34.csv',
}


In [271]:
#task.dict['parcing']='results//_reserve_SL_diamonds2022-09-07-08-07.csv'

In [53]:
task.save()

не правильный словарь
dict записан:  d_SL_diamonds2022-09-07-17-29_2022-09-07-17-34.json


In [75]:
t=Dtask({})

установка : name2022-09-07-12-52


In [76]:
t.load()

[(0, 'd_SL_diamonds2022-09-06-22-58_2022-09-06-23-02.json'), (1, 'd_SL_diamonds2022-09-07-06-31_2022-09-07-06-34.json'), (2, 'd_SL_diamonds2022-09-07-08-24_2022-09-07-08-25.json'), (3, 'd_SL_diamonds2022-09-07-12-41_2022-09-07-12-51.json'), (4, 'd_SL_diamonds_2022-09-07-08-07.json')]
введите номер файла: 3
dict загружен :tasklogs/d_SL_diamonds2022-09-07-12-41_2022-09-07-12-51.json,
 всего        
 загружены атрибуты dict_keys(['name', 'domain', 'urls0', 'urlslast', 'descr', 'datestr0', 'datestr', 'links', 'parcing'])         
надо определить имя, из какого атрибута взять? 
        [(0, 'name'), (1, 'domain'), (2, 'urls0'), (3, 'urlslast'), (4, 'descr'), (5, 'datestr0'), (6, 'datestr'), (7, 'links'), (8, 'parcing')]
номер: 0
выбрано - SL_diamonds2022-09-07-12-41


In [77]:
t.print()

task name: SL_diamonds2022-09-07-12-41
links len =  120 

{'name': 'SL_diamonds2022-09-07-12-41',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-3/',
'descr': '',
'datestr0': '2022-09-07-12-41',
'datestr': '2022-09-07-12-51',
'parcing': 'results//_reserve_SL_diamonds2022-09-07-12-412022-09-07-12-51.csv',
}


## Добываем линки на карточки SUNLIGHT 

In [ ]:
#taskname = 'SL_всеЮИ'
#taskdescr = '579 стр с фильтром https://sunlight.net/catalog/?material=326,327,328,329,3603,632&product_type=317,319,365,366,373,4,414,478,493,494,5,541,745,8,890'
#urls = 'https://sunlight.net/catalog/?material=326,327,328,329,3603,632&product_type=317,319,365,366,373,4,414,478,493,494,5,541,745,8,890'
#domain = f'https://sunlight.net'

In [ ]:
#links = []

In [ ]:

links += getlinks_sunl(urls)
for i in range(512,598):
    #pages = f'page-{i}/'#все 
    pages = f'&page={i}'#без вставки

    print(i, urls+pages)
    links += getlinks_sunl(urls+pages)
    

In [ ]:
len(links)

In [ ]:
links = list(set(links))

In [ ]:
d2 = Dlinks()

In [ ]:
d2.__doc__

In [ ]:
taskname,  urls, domain, taskdescr

In [ ]:
d2.save(taskname, links, urls, domain, taskdescr)

In [ ]:
links = []

In [ ]:
#def main():

#urls = f'https://sunlight.net/catalog/'#все

urls = f"https://sunlight.net/catalog/?stone=1" #без вставки
domain = f'https://sunlight.net'
links += getlinks_sunl(urls)
for i in range(2,148):
    #pages = f'page-{i}/'#все 
    pages = f'&page={i}'#без вставки

    print(i, urls+pages)
    links += getlinks_sunl(urls+pages)
    

In [ ]:
len(links)

In [ ]:
#drop duplicates subset keep
links = list(set(links))

In [ ]:
domain+links[0]

## Словарь линков (save-load)

In [ ]:
import json

In [ ]:
import datetime
datestr = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
datestr

In [ ]:
datestr

In [ ]:
#Параметры задачи
taskname, taskdescr =  'бескаменка300822', 'все БК  8810'

In [ ]:
#Параметры линка
datestr, domain, links

In [ ]:
#словарь для сохранения
dlinks = {'taskname':taskname, 
           'links': links,
           'datestr':datestr,
           'urls':urls,
           'domain':domain,
           'taskdescr':taskdescr}

In [ ]:
with open(folderlink+'dlinks_sunlight'+datestr+'.json', 'w') as writefile:
    json.dump(dlinks, writefile)

In [ ]:
dlinks.keys()

In [ ]:
folderlinks = 'links/'
files = os.listdir(folderlink)
print([e for e in enumerate(files)])

In [ ]:
with open(folderlinks+files[0], 'r') as writefile:
    dlinks = json.load(writefile)

In [ ]:
dlinks

In [ ]:
links = dlinks[taskname][0]
len(links)

In [ ]:
dlinks['domain']+dlinks['links'][1]

In [ ]:
folderlink

### Сборка функции или объекта

In [ ]:
class Dlinks():
    '''taskname, links, urls, domain, taskdescr, datestr=None'''
    #import datetime
    import json
    import os
    folderlinks = 'links/'
    
    def save(self, taskname, links, urls, domain, taskdescr, datestr=None):
        self.taskname = taskname
        self.links = links
        self.urls = urls
        self.domain = domain
        self.taskdescr = taskdescr
        if datestr:
            self.datestr = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        else:
            self.datestr = datestr
        dlinks = {'taskname':taskname, 
           'links': links,
           'datestr':datestr,
           'urls':urls,
           'domain':domain,
           'taskdescr':taskdescr}
        
        with open(self.folderlinks+'dlinks_'+self.taskname+'_'+datestr+'.json', 'w') as writefile:
                   json.dump(dlinks, writefile)
        print('линки записаны: ', self.folderlinks+'dlinks_'+self.taskname+'_'+datestr+'.json')
        
        
    
    def load(self):
        files = os.listdir(self.folderlinks)
        print([e for e in enumerate(files)])
        i = int(input('введите номер файла: '))
        with open(self.folderlinks + files[i], 'r') as writefile:
            dlinks = json.load(writefile)
        self.taskname = dlinks['taskname']
        self.links = dlinks['links']
        self.urls = dlinks['urls']
        self.domain = dlinks['domain']
        self.taskdescr = dlinks['taskdescr']
        self.datestr = dlinks['datestr']
        print(f'линки загружены с :{self.folderlinks + files[i]},\n всего\
        {len(self.links)} линков,\n загружены атрибуты {dlinks.keys()} \
        {self.taskname, self.urls, self.domain, self.taskdescr, self.datestr}')
        return self.links    
        

In [ ]:
taskname, urls, domain, taskdescr, datestr = ('бескаменкаSL',
 'https://sunlight.net/catalog/?stone=1',
 'https://sunlight.net',
 'все БК  8810',
 '2022-08-30-11-18')

In [ ]:
taskname, urls, domain, taskdescr, datestr

In [ ]:
olinks = Dlinks()

In [ ]:
olinks?

In [ ]:
olinks.save(taskname, links, urls, domain, taskdescr, datestr )

In [ ]:
links = olinks.load()

In [ ]:
links

In [ ]:
olinks.domain

In [ ]:
olinks.datestr

## Разработка парсера по карточкам

In [ ]:
#urls = 'https://sunlight.net/catalog/page-3/'

In [ ]:
mysoup(urls)

In [ ]:
#def getlinks(urls):
quotes = mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")

In [ ]:
len(quotes)

In [ ]:
quotes[1].get("href")

In [ ]:
res = [q.get("href") for q in quotes]
#    print(len(res), 'links is ready ' , res[0], '...', res[-1])
#    return res
    

In [ ]:
res

In [ ]:
<a href="/catalog/pendants_90939.html" class="cl-item-link js-cl-item-link js-cl-item-root-link" itemprop="url">
                    <span itemprop="name">Подвеска, вставка: фианит; наносапфир; Серебро 925 пробы.</span>
                </a>


In [ ]:
<a href="/catalog/ring_47781.html" class="cl-item-link js-cl-item-link js-cl-item-root-link" itemprop="url">
                    <span itemprop="name">Кольцо, вставка: фианит; Серебро 925 пробы.</span>
                </a>

## Собранная функция парсера по карточкам

### Версия 29-08-2022 SUNLIGHT

In [ ]:
data = pd.DataFrame(columns=['h1','art','price','price2','gold','gold2','weight','gems','gems2','url'])


In [ ]:

def parcing(links): 

    for i in range(3000):#links)):
        try:
            row = len(data)
            urlcard = domain +links[i]
            card = mysoup(urlcard)
            data.loc[row,'url'] = urlcard
            data.loc[row,'h1'] = card.find("h1").text.strip()
            data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
            data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
            data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
            a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
            data.loc[row,'gold'] = ';'.join([x.text for x in a])
            data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
            data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
            #print(i, urlcard)
        except (AttributeError,TypeError) as e:
            print('error',e)
        except IndexError:
            print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
        print(i, urlcard)
    
def byrepost():

    def gold(t,patt):
        f = re.search(patt,t)
        return f.group() if f else 're не найдено'
    
    pattern_gold = r'[Сс]ереб\w+|[Зз]олот\w*|[Сс]таль'
    pattern_diam = r'[Бб]риллиант\w*'
    pattern_probe = r'[\d]{,3}(?=\s*проб)'
    pattern_gramm = r'[\d.\d]{,5}(?=г;)'
    pattern_vst = r'вставк\w*'#r'вставк\w*'
    pattern_type = r'(?<=;)\w+\s?\w*'        
        
    
    data['diam'] = data.gold.apply(lambda x: gold(x, pattern_diam))
    data['goldsilv'] = data.h1.apply(lambda x: gold(x, pattern_gold))
    data['probe'] = data.gold.apply(lambda x: gold(x, pattern_probe))
    data['gramm'] = data.gold.apply(lambda x: gold(x, pattern_gramm))
    data['vst'] = data.gold.apply(lambda x: gold(x, pattern_vst))
    data['type'] = data.gold.apply(lambda x: gold(x, pattern_type))

In [ ]:
parcing(links)

In [ ]:
byrepost()

In [ ]:
data

In [ ]:
#data = pd.DataFrame(columns=['h1','art','price','price2','gold','gold2','weight','gems','gems2','url'])

In [ ]:
#29-08-2022 for sunlight 6000
#25-08-2022 all
#06-07-2022 серебро
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(len(links)):
    try:
        row = len(data)
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
        data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
        data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
        a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
        data.loc[row,'gold'] = ';'.join([x.text for x in a])
        data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
        data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
        #print(i, urlcard)
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)


In [ ]:
### Поиск ошибки
for i in range(4216, 4217):
    try:
        row = len(data)
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
        data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
        data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
        a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
        data.loc[row,'gold'] = ';'.join([x.text for x in a])
        data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
        data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
        #print(i, urlcard)
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)

In [ ]:
len(data)

In [ ]:
data

In [ ]:
domain+links[0]

In [ ]:
urlcard = domain +links[100]
card = mysoup(urlcard)


In [ ]:
#collect 29-08-22
row = len(data)
data.loc[row,'url'] = urlcard
data.loc[row,'h1'] = card.find("h1").text.strip()
data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
data.loc[row,'gold'] = ';'.join([x.text for x in a])
data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])

In [ ]:
data

In [ ]:
data.gold.loc[10]

## Разработка rе

In [ ]:
pattern_gold = r'[Сс]ереб\w+|[Зз]олот\w*|[Сс]таль'
pattern_diam = r'[Бб]риллиант\w*'
pattern_probe = r'[\d]{,3}(?=\s*проб)'
pattern_gramm = r'[\d.\d]{,5}(?=г;)'
pattern_vst = r'вставк\w*'#r'вставк\w*'
pattern_type = r'(?<=;)\w+\s?\w*'
def gold(t,patt):
    f = re.search(patt,t)
    return f.group() if f else 're не найдено'

In [ ]:
t = 'Артикул: 114165;Кольцо    \nБелое золото 585 пробы.;Вес изделия: 3.04г;Кольцо    \nБелое золото 585 пробы.'
re.search(pattern_type,t)

In [ ]:
patt = pattern_type
#patt = pattern_diam

In [ ]:
patt

In [ ]:
i = 1
gold(data.gold.loc[i],patt), data.gold.loc[i]

In [ ]:
data.gold.apply(lambda x: gold(x, patt)).unique()

In [ ]:
data.gold.apply(lambda x: gold(x, patt)).value_counts()

In [ ]:
data[data.gold.apply(lambda x: gold(x, patt))=='Ножной']

## Результат первичной обработки 

In [ ]:
#PCH01-09-2022
def byrepost(data):
    def gold(t, patt):
        f = re.search(patt, t)
        return f.group() if f else 're не найдено'

    pattern_gold = r'[Сс]ереб\w+|[Зз]олот\w*|[Сс]таль'
    pattern_diam = r'[Бб]риллиант\w*'
    pattern_probe = r'[\d]{,3}(?=\s*проб)'
    pattern_gramm = r'[\d.\d]{,5}(?=г;)'
    pattern_vst = r'вставк\w*'  # r'вставк\w*'
    pattern_type = r'(?<=;)\w+\s?\w*'

    data['diam'] = data.gold.apply(lambda x: gold(x, pattern_diam))
    data['goldsilv'] = data.h1.apply(lambda x: gold(x, pattern_gold))
    data['probe'] = data.gold.apply(lambda x: gold(x, pattern_probe))
    data['gramm'] = data.gold.apply(lambda x: gold(x, pattern_gramm))
    data['vst'] = data.gold.apply(lambda x: gold(x, pattern_vst))
    data['type'] = data.gold.apply(lambda x: gold(x, pattern_type))

In [ ]:
###

In [ ]:
data = pd.concat([data,data3000_8000], ignore_index=True)

In [ ]:
#res

    data['diam'] = data.gold.apply(lambda x: gold(x, pattern_diam))

    data['goldsilv'] = data.h1.apply(lambda x: gold(x, pattern_gold))

    data['probe'] = data.gold.apply(lambda x: gold(x, pattern_probe))

    data['gramm'] = data.gold.apply(lambda x: gold(x, pattern_gramm))

    data['vst'] = data.gold.apply(lambda x: gold(x, pattern_vst))

    data['type'] = data.gold.apply(lambda x: gold(x, pattern_type))

In [ ]:
#разработка

In [ ]:
<div class="supreme-product-card-description__item-title">
    Бренд
  </div>

In [ ]:
card.find_all('span', class_="supreme-product-card-description__item-text")[4].text

In [ ]:
<a href="https://sunlight.net/catalog/sergey-gribnyakov.html" class="supreme-product-card-description__item-text">SERGEY GRIBNYAKOV</a>

In [ ]:
card.find_all('a', class_="supreme-product-card-description__item-text")

In [ ]:
<div class="supreme-product-card__product-article-text">
      Артикул: 165495
    </div>

In [ ]:
card.find('div', class_="supreme-product-card__product-article-text").text.strip()

In [ ]:
<div class="supreme-product-card__price-discount-price">
  5 395&nbsp;₽
</div>

In [ ]:
card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')

In [ ]:
card.find('div', class_="supreme-product-card__price-discount-price")

In [ ]:
<div class="supreme-product-card__price-old">
  26 975&nbsp;₽
</div>

In [ ]:
<p class="supreme-product-card__description supreme-product-card__description_default">Кольцо «Бриллианты Якутии» с 21 бриллиантом, огранка круг 17 граней, 0.04 карат, цвет 2, чистота 2, тип огранки Б; 
Розовое золото 585 пробы</p>

In [ ]:
a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")

In [ ]:
a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
';'.join([x.text for x in a])

## Визуализация

In [ ]:
 urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        data.loc[row,'price2'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = card.find_all('span','href')[2].text.strip()
        data.loc[row,'gold2'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems2'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]

In [ ]:
card.find_all('span','href')[2].text.strip()

In [ ]:
div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
#data.loc[row,'gems'] = 
re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)

In [ ]:
#25-08-2022 all
#06-07-2022 серебро
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(len(links)):
    try:
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        data.loc[row,'price2'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = card.find_all('span','href')[2].text.strip()
        data.loc[row,'gold2'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems2'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
        
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)


In [ ]:
data

In [ ]:
data.to_excel('pars_16-06-2022_muiz_RSE.xlsx')

### Версия 14-04-2022

In [ ]:
data = pd.DataFrame(columns=['h1','art','price','gold','weight','gems','url'])

In [ ]:
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(len(links)):
    try:
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        #data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)


## save res

In [ ]:
import os

In [ ]:
folders = os.listdir()
[q for q in (enumerate(folders))]

In [ ]:
resfolder = folders[26]


In [ ]:
resfolder

In [ ]:
data.to_excel(resfolder+'/'+'sunlight_test01092022.xlsx')


In [ ]:
files = os.listdir('results/')
[f for f in enumerate(files)]

In [ ]:
data = pd.read_csv('results/'+files[9])

In [ ]:
data.loc[1200,'h1']= 1111

In [ ]:
data

In [ ]:
#06-07-2022 серебро
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(0,5):# len(links)):
    try:
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        #data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = card.find_all('span','href')[2].text.strip()
        #data.loc[row,'gold'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        #div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        #data.loc[row,'gems'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
        
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)

In [ ]:
import os

In [ ]:
def fold(sourcepath):
    folders = os.listdir(sourcepath)
    print(*[nf for nf in enumerate(folders)] )
    case = input (sourcepath)
    if case.isdigit():
        sourcepath += folders[int(case)]+"/"
        sourcepath = fold(sourcepath)
        return sourcepath
    else: 
        print(sourcepath)
        return sourcepath


In [ ]:
def source():
    
    p0 = input('path UR-JVV/Users/DAS/Desktop/GM TMP/A Переоценка/ y or enter')
    sourcepath = fold(r'//UR-JVV/Users/DAS/Desktop/GM TMP/A Переоценка/' if p0 == 'y' else p0)
    #'for_site2-1922-1.xlsx'
    files =os.listdir(sourcepath)
    
    filename = files[int(input([n for n in enumerate(files)]))]
    print('filename',filename)
    if not input("y?")=='y':
        print('aborded')
        return False
    #df, file_in = load_df(sourcepath+filename)
    print('test columns in ', (sourcepath+filename))
    return sourcepath, filename
#E:/OM2022/PEREOZENKA/ГИИС

In [ ]:
sourcepath, filename = source()


In [ ]:

sourcepath, filename

In [ ]:
df = pd.read_csv(sourcepath+filename)#.drop(columns='Unnamed: 0')

In [ ]:
df

## Загрузка обработка

In [ ]:
df = pd.read_excel('results/mius_серебро790_06072022.xlsx').drop(columns='Unnamed: 0')

In [ ]:
df.columns

In [ ]:
df.dropna(subset=['weight'], inplace = True)

In [ ]:
#t = 'Серьги c фианитами E2029-1210015246-Z'
#t = 'Кольцо c фианитами R4150-S-LR19653'
t2 = 'Кольцо c жемчугом, сапфирами и фианитами R4150-S-LRP34769S'
t = 'Кольцо fdfd'

In [ ]:
patt_h1_0 = r'([\w\W]+)(?= [A-Z]\d)+?'

In [ ]:
 r'([\w\W]+)(?= [A-Z]\d)+?'

In [ ]:
re.search(patt_h1_0, t2)[1].strip()

In [ ]:
re.search( r'([\w\W]+)', t)[1].strip()

In [ ]:
df['h1_0'] = df.h1.apply(lambda x: re.search(patt, x)[1] if re.search(patt, x) else x)

In [ ]:
patt = patt_h1_0
for n,x in enumerate(df.h1):
    print(n, x,' -- ', (lambda x: re.search(patt, x)[1] if re.search(patt, x) else x)(x))#.strip())

In [ ]:
str(patt_h1_0)

In [ ]:
df.loc[:]

In [ ]:
df['gemsstr'] = df['gems'].str.replace('\[|\]','').str.strip("'")

df['gemslist'] = df.gemsstr.str.split(';')

In [ ]:
df['gemslist'].map(len).unique()

In [ ]:
df['lengem'] = df['gemslist'].map(len)

In [ ]:
df.loc[df['gemslist'].map(len) <= 2, ['gems','gemslist','lengem']]

In [ ]:
sum(df['gemslist'].map(len) > 2)

In [ ]:
df['gem01'] = df['gemslist'].apply(lambda x: x[0])

In [ ]:
df.columns

In [ ]:
df01 = df.loc[df['gemslist'].map(len) <= 2, ['h1_0','h1','gems', 'price', 'gold', 'weight','gem01', 'url','lengem']]

In [ ]:
df01

In [ ]:
t = '38 Фианитов огранки "Круглая", 0.34 карат'

In [ ]:
dic={}

In [ ]:
re.match(r'\d+',t)[0] 
dic['шт_вст'] = r'\d+'

In [ ]:

dic['карат_вст'] = r'[\d.\d]+(?= карат)'
re.search(dic['карат_вст'],t)[0]

In [ ]:
dic['описание_вст'] = r'(?!\d+).+(?=,)'
re.search(dic['описание_вст'],t)[0]

In [ ]:
for c in dic.items():
    print(c)
    df01[c[0]] = df01.gem01.apply(lambda x: re.search(c[1],x)[0] if re.search(c[1],x) else '')

In [ ]:
for n, x in enumerate(df01.gem01):
    print(n,x,'---> ', re.search(r'[\d.\d]+(?= карат)',x)[0] if re.search(r'[\d.\d]+(?= карат)',x) else '')

In [ ]:
df01

In [ ]:
resfolder

In [ ]:
df01.to_excel(resfolder+'/'+'нефть1сорт_серебро_мюз_604_06-07-2022.xlsx')

In [ ]:
df['IDizd'] = df.index

In [ ]:
df.columns

In [ ]:
df.explode("gemslist")  #[["IDizd","gemslist"]]

### Тест работы from classes_parsing import Dlinks

In [ ]:
from classes_parsing import Dlinks

In [ ]:
dir(Dlinks)# Dlinks.__dir__(Dlinks)

In [ ]:
Dlinks.__doc__

In [ ]:
taskname, urls, domain, taskdescr, datestr

In [ ]:
d = Dlinks()

In [ ]:
d.folderlinks

In [ ]:
z = print()

In [ ]:
z == None

In [ ]:
if not z:
    #self.datestr = 
    print(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M'))
else:
    print(z)

In [ ]:
d.save('testDlink', links[:10], urls, domain, 'проверка')

In [ ]:
d.taskdescr, d.taskname, d.datestr

In [ ]:
self = d

In [ ]:
self.folderlinks + 'dlinks_' + self.taskname + '_' + datestr + '.json'

### Отступление, о том как это превратилось в текст и когда, и как я обратно в лист

In [ ]:
from bs4 import BeautifulSoup
import requests
from requests import Request, Session
import pandas as pd

In [ ]:
import re

In [ ]:
def mysoup(urls):
    response = requests.get(urls)
    return BeautifulSoup(response.text, 'lxml')   

In [ ]:
card = mysoup('https://miuz.ru/catalog/rings/R4150-S-LRP34775CP/')

In [ ]:
div_gems = card.select('body > div.grid > div > \
div > div.detail__item.detail__item--new > \
div.detail__item-col._3.position-relative >\
form > div:nth-child(8)')
a = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
a, type(a)

In [ ]:
data = pd.DataFrame(columns=['h1','art','price','gold','weight','gems','url'])
data.loc[len(data),'gems'] = a

In [ ]:
data['gems']

In [ ]:
data.loc[0,'gems'], type(data.loc[0,'gems'])

In [ ]:
data.to_excel('data.xlsx', index=None)

In [ ]:
data_e = pd.read_excel('data.xlsx')

In [ ]:
data_e['gems']

In [ ]:
data_e.loc[0,'gems'] , type(df.loc[0,'gems'])

In [ ]:
data_e['gems_list'] = data_e['gems'].str.replace('\[|\]','').str.strip("'").str.split(';')

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
data_e[['gems','gems_list']]

In [ ]:
data_e.loc[0,'gems_list'] , type(data_e.loc[0,'gems_list'])

In [ ]:
### Отступление, о том как это превратилось в текст и когда, и как я обратно в лист

from bs4 import BeautifulSoup
import requests
from requests import Request, Session
import pandas as pd

import re

def mysoup(urls):
    response = requests.get(urls)
    return BeautifulSoup(response.text, 'lxml')   

card = mysoup('https://miuz.ru/catalog/rings/R4150-S-LRP34775CP/')

div_gems = card.select('body > div.grid > div > \
div > div.detail__item.detail__item--new > \
div.detail__item-col._3.position-relative >\
form > div:nth-child(8)')
a = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
a, type(a)

data = pd.DataFrame(columns=['h1','art','price','gold','weight','gems','url'])
data.loc[len(data),'gems'] = a

data['gems']

data.loc[0,'gems'], type(data.loc[0,'gems'])

data.to_excel('data.xlsx', index=None)

data_e = pd.read_excel('data.xlsx')

data_e['gems']

data_e.loc[0,'gems'] , type(data_e.loc[0,'gems'])

data_e['gems_list'] = data_e['gems'].str.replace('\[|\]','').str.strip("'").str.split(';')

pd.options.display.max_colwidth = 200

data_e[['gems','gems_list']]

data_e.loc[0,'gems_list'] , type(data_e.loc[0,'gems_list'])

In [ ]:
df

In [ ]:
df.drop(columns='gemsl', inplace=True)

In [ ]:
s = "one [two] three [1] [2]"
print(re.sub('\[|\]', '', s))

# NOTE

In [ ]:
#Нужная - перезапуск модулей и классов
import importlib 

import classes_parsing #https://stackoverflow.com/questions/6946376/how-to-reload-a-class-in-python-shell

importlib.reload(classes_parsing)

from classes_parsing import Dlinks

classes_parsing.Dlinks.__doc__

Dlinks.__doc__

In [ ]:
for i in range(6505):
    if i/500 in range(100):
        print(i)

In [ ]:
linkspd = pd.DataFrame(links)

In [ ]:
linkspd.rename(columns={0:'short'}, inplace=True)

In [ ]:
linkspd['path'] = domain +linkspd['short']

In [ ]:
linkspd.to_excel(resfolder+'/'+'links4154_2410522.xlsx')

In [ ]:
re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))

In [ ]:
card = mysoup(urlcard)
row = len(data)
data.loc[row,'h1'] = card.find("h1").text.strip()
data.loc[row,'art'] = card.select('body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')

data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
              

#quotes_inner = card.find_all('div', "detail__item-option")
#quotes_inner

data.loc[row,'gold'] = re.findall(r"(Золото\s+\d+)",str(quotes_inner[0]))[0]

data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])

div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')

data.loc[row,'gems'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
